# Import des modules

In [1]:
import glob
import pandas as pd

# Lecture des données 

Données venant du Sirsé et cartosanté

In [2]:
liste = glob.glob('*.csv')
for i in range(len(liste)):
    if i == 0 :
        db = pd.read_csv(liste[i], skiprows=2, sep=';', encoding="utf-8", low_memory=False)
    else:
        db_temp = pd.read_csv(liste[i], skiprows=2, sep=';', encoding="utf-8", low_memory=False)
        try:
            db_temp.drop('Libellé',axis=1, inplace=True)
        except:
            pass
        db = db.merge(db_temp, left_on='Code', right_on='Code', how='inner')

Données de l'insee sur les codes de départements et regions

In [3]:
dep = pd.read_csv("../dep/communes-departement-region.csv", sep=",", encoding="utf-8")
#dep = dep.groupby(['code_commune_INSEE']).count().reset_index()
dep['code_commune_INSEE'] = dep['code_commune_INSEE'].apply(lambda x: '0'+str(x) if(len(str(x)) == 4) else str(x))
dep['code_postal'] = dep['code_postal'].apply(lambda x: '0'+str(x) if(len(str(x)) == 4) else str(x))
dep['code_dep'] = dep['code_commune_INSEE'].str[:2]
dep = dep[['code_dep','code_commune_INSEE', 
     'nom_departement','nom_region','code_postal',
     'code_region','longitude','latitude']]

# Modification des noms des colonnes

In [4]:
db.columns = ['Code','Nom_com', 'NG19_60>', 'NG19T',
              'Pop16_<18', 'NC16_15_64','NC16_15_24',
              'NRP16','RFM16','Pop16_>65','TN16','PM16']

# Jointure des données

In [5]:
merged_data = pd.merge(db,dep, left_on= "Code", right_on ="code_commune_INSEE", how="left")
data = pd.DataFrame(merged_data['Code'].value_counts() == 1)
liste_code_commune = data[data['Code']==True].index
merged_data_one = merged_data[merged_data['Code'].isin(liste_code_commune)]
#merged_data_one['code_region'] = merged_data_one['code_region'].astype(int).astype(str)
merged_data_one.drop(["Code","longitude","latitude"], axis=1, inplace=True)

C:\Users\joaka\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [12]:
merged_data_one.to_csv("output_data.csv")

# Vue sur les données

In [11]:
merged_data_one.columns

Index(['Nom_com', 'NG19_60>', 'NG19T', 'Pop16_<18', 'NC16_15_64', 'NC16_15_24',
       'NRP16', 'RFM16', 'Pop16_>65', 'TN16', 'PM16', 'code_dep',
       'code_commune_INSEE', 'nom_departement', 'nom_region', 'code_postal',
       'code_region'],
      dtype='object')

In [9]:
merged_data_one.describe()

,NG19_60>,NG19T,Pop16_<18,NC16_15_64,NC16_15_24,NRP16,RFM16,Pop16_>65,TN16,PM16,code_region
count,33217.000000,33217.000000,33203.000000,33203.000000,33203.000000,33123.000000,29433.000000,33203.000000,33197.000000,33203.000000,33215.000000
mean,0.402354,1.290785,345.717194,96.305695,20.686354,92.887571,20809.415044,296.759871,9.352649,1554.596151,52.258739
std,2.232292,6.025696,1246.816357,447.360562,91.649966,661.855511,3638.465890,985.375680,6.647526,5822.401142,24.295222
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7570.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,0.000000,39.000000,9.000000,1.000000,0.000000,18504.000000,39.000000,5.400000,191.000000,28.000000
50%,0.000000,0.000000,95.000000,20.000000,4.000000,0.000000,20240.000000,83.000000,9.000000,431.000000,44.000000
75%,0.000000,0.000000,241.000000,51.000000,11.000000,14.000000,22426.000000,206.000000,12.500000,1065.000000,76.000000
max,123.000000,312.000000,38597.000000,16193.000000,2931.000000,32624.000000,50092.000000,41271.000000,200.000000,233484.000000,94.000000


# Séparation des données

val_descriptive = 

# Calcule des valeurs à afficher

In [6]:
def calcule_indicateur(df):
    colnames = df.columns
    for colname in colnames:
        df[colnames] = df[colnames].apply(lambda x: '0'+str(x) if(len(str(x)) == 4) else str(x))